In [8]:
# Imports
import kfp
import json, os, shutil
import kfp.dsl as dsl
from kfp.dsl import component, ContainerOp
from kfp.dsl.types import Dict, List
import kfp.compiler as compiler
from kfp.components import InputPath, InputTextFile, InputBinaryFile, OutputPath, OutputTextFile, OutputBinaryFile
from kubernetes.client.models.v1_local_object_reference import V1LocalObjectReference

In [10]:
# STEP 1
@component
def clean(message:str):
    return ContainerOp(
        name = 'test1',  
        image = 'docker.yq.credo.be/cust/yq-python-ml-notebook:v2.5.6.1', 
        command = ['python', 'scripts/simplest_pipeline/step1.py'], 
        arguments =['--message', message]
    ).add_pod_label('yq-environment', 'true')

In [11]:
# STEP 2
@component
def generate(scenarios:int):
    return ContainerOp(
        name = 'scenarios_',  
        image = 'docker.yq.credo.be/cust/yq-python-ml-notebook:v2.5.6.1', 
        command = ['python', 'scripts/simplest_pipeline/step2.py'], 
        arguments =[ '--scenarios', scenarios ]
    ).add_pod_label('yq-environment', 'true')

In [13]:
# BONDS VaR PIPELINE
runName = 'Bohdan_Test_Pipeline'
@dsl.pipeline(name='Bonds Test')
def bonds_pipeline(scenarios:int=100,
                   pods:int=1,
                   message:str="message from pipeline"):
    
    step05 = clean(message)
    
    # Kubeflow Garbage Collection 
    #dsl.get_pipeline_conf().set_ttl_seconds_after_finished(86400)
    dsl.get_pipeline_conf().set_ttl_seconds_after_finished(720)
    
    with dsl.ParallelFor(pods) as pod:
        step_1 = generate(scenarios=scenarios).after(step05).set_memory_limit('0.3G').set_cpu_request('0.3')

In [14]:
# with this one you can run it directly from notebook. Follow the "here" buttons below
if __name__ == '__main__':
    kfp.Client().create_run_from_pipeline_func(bonds_pipeline, arguments={}, run_name=runName, namespace='bohdana')

In [7]:
# use this when you want to compile and download your pipeline
if __name__ == '__main__':
    compiler.Compiler().compile(bonds_pipeline, 'scripts/BV_111P.yaml', type_check=True) 